In [3]:
# libraries
import pandas as pd
import numpy as np
import os
from statsmodels.tsa.stattools import acf, adfuller
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.svm import SVR
from tqdm import tqdm
from collections import defaultdict

# read in data

data downloaded from:

 - https://zenodo.org/record/5946808#.ZGNpddbP23I
 - https://zenodo.org/record/5841834#.ZGNpTNbP23J

### Wind data

##### Status data

In [4]:
# read in files
turbine_status = pd.read_csv('../data/Windturbinen/Kelmarsh/Status_Kelmarsh_1_2016-01-03_-_2017-01-01_228.csv',
                             skiprows = 9)

In [5]:
print(turbine_status.columns)
turbine_status.head()

Index(['Timestamp start', 'Timestamp end', 'Duration', 'Status', 'Code',
       'Message', 'Comment', 'Service contract category', 'IEC category'],
      dtype='object')


,Timestamp start,Timestamp end,Duration,Status,Code,Message,Comment,Service contract category,IEC category
0,2016-01-14 19:28:03,2016-01-23 14:36:32,211:08:29,Stop,111,Emergency stop nacelle,NaN,Emergency stop switch (Nacelle) (11),Forced outage
1,2016-01-14 19:28:03,2016-01-14 19:38:03,00:10:00,Warning,5720,Brake accumulator defect,NaN,Warnings (27),NaN
2,2016-01-14 19:28:05,2016-01-23 11:27:46,207:59:41,Informational,3835,Cable panel breaker open,NaN,Warnings (27),NaN
3,2016-01-14 19:28:05,2016-01-23 11:27:46,207:59:41,Informational,3830,Supply circuit breaker earthed,NaN,Warnings (27),Full Performance
4,2016-01-14 19:28:05,2016-01-23 14:09:18,210:41:13,Warning,3870,Overload transformer fan outlet air,NaN,Warnings (27),Full Performance


Status data is not relevant to this thesis !

##### Turbine Data

In [7]:
def reading_Windturbines (turbine_directory):
    # Columns to keep
    columns_to_keep = [
        '# Date and time',
        'Wind speed (m/s)',
        'Long Term Wind (m/s)',
        'Energy Export (kWh)'
    ]

    # Directory containing CSV files
    directory = f'../data/Windturbinen/{turbine_directory}/'

    # Dictionary to hold DataFrames for each turbine
    turbine_dataframes = defaultdict(list)

    # Get a list of CSV files in the directory
    csv_files = [f for f in os.listdir(directory) if f.startswith(f"Turbine_Data_{turbine_directory}_") and f.endswith(".csv")]

    # Iterate through the files in the directory with a tqdm progress bar
    for filename in tqdm(csv_files, desc='Processing files'):
        # Extract the turbine number from the filename
        turbine_number = filename.split("_")[3]  # Assuming the number is in this position

        # Read the CSV file, skipping the first 9 rows
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath, skiprows=9, usecols=columns_to_keep)

        # Convert the "Date and time" column to datetime
        df['# Date and time'] = pd.to_datetime(df['# Date and time'])

        # Append the DataFrame to the appropriate turbine's list
        turbine_dataframes[turbine_number].append(df)

    # Concatenate the DataFrames for each turbine
    for turbine_number, dfs in turbine_dataframes.items():
        turbine_dataframes[turbine_number] = pd.concat(dfs)
        turbine_dataframes[turbine_number].sort_values('# Date and time', inplace=True)

    # Print the keys for the dictionary
    print("\n dictionary keys:")
    print(turbine_dataframes.keys())
    # print descriptive stuff for exemplary key
    print('\n Information for exemplary key:')
    first_key = list(turbine_dataframes.keys())[0]
    print('shape')
    print(turbine_dataframes[first_key].shape)
    print('\n missing values')
    print(turbine_dataframes[first_key].isna().sum())

    return turbine_dataframes

In [8]:
Kelmarsh_df = reading_Windturbines('Kelmarsh')

Processing files: 100%|██████████| 36/36 [00:18<00:00,  1.94it/s]


 dictionary keys:
dict_keys(['6', '4', '1', '3', '2', '5'])

 Information for exemplary key:
shape
(288864, 4)

 missing values
# Date and time            0
Wind speed (m/s)        9223
Long Term Wind (m/s)       0
Energy Export (kWh)     5072
dtype: int64


In [9]:
Penmanshiel_df = reading_Windturbines('Penmanshiel')

Processing files: 100%|██████████| 84/84 [00:45<00:00,  1.84it/s]


 dictionary keys:
dict_keys(['07', '08', '02', '05', '06', '15', '10', '14', '01', '04', '11', '12', '13', '09'])

 Information for exemplary key:
shape
(267014, 4)

 missing values
# Date and time            0
Wind speed (m/s)        5881
Long Term Wind (m/s)       0
Energy Export (kWh)     1032
dtype: int64


### FlexGuide Data

In [10]:
# merge files
ENIT_df = pd.concat([ENIT_2022_1, ENIT_2022_2, ENIT_2023_1])

# rename columns
ENIT_df.columns = ENIT_df.columns.str.replace('Wirkarbeit (Bezug) ', '').str.strip()

# add residual columns
ENIT_df['1.8 - Residual'] = ENIT_df.iloc[:, 1] - ENIT_df.iloc[:, 2:8].sum(axis=1)
ENIT_df['2.7 - Residual'] = ENIT_df.iloc[:, 9] - ENIT_df.iloc[:, 10:15].sum(axis=1)
ENIT_df['0.1 - Residual'] = ENIT_df.iloc[:, 16] - ENIT_df.iloc[:, 1] - ENIT_df.iloc[:, 9]

NameError: name 'ENIT_2022_1' is not defined

In [ ]:
# Calculate the mean and standard deviation for each column
mean = ENIT_df.iloc[:, 1:15].mean()
std_dev = ENIT_df.iloc[:, 1:15].std()

# Identify outliers using twice the standard deviation
outliers = (ENIT_df.iloc[:, 1:15] < (mean - 2 * std_dev)) | (ENIT_df.iloc[:, 1:15] > (mean + 2 * std_dev))

# Print the number of True values in each column
print("Number of outliers in each column:")
print(outliers.sum())

# Get the row and column indices of the True values
true_values_indices = outliers.where(outliers).stack().index

# Print the row and column indices of the True values
print("\nIndices of outliers:")
for row, col in true_values_indices:
    print(f"Row: {row}, Column: {col}")


Number of outliers in each column:
1.0 - Trafo 1 [Wh]                                   0
1.1 - Neubau, Wohnhaus, Holzplatz [Wh]               0
1.2 - Halle 3/1 Absaugung [Wh]                       0
1.3 - Halle 4/2 Maschinensaal/Tischfertigung [Wh]    0
1.4 - Halle 2/2 Verwaltung, Entwicklung [Wh]         0
1.5 - Halle 4/5 Lackieranlage [Wh]                   0
1.6 - Halle 1/6 Hausmeister [Wh]                     0
1.7 - Halle 3/3 Kompressor, Stuhlmontage [Wh]        0
2.0 - Trafo 2 [Wh]                                   0
2.1 - Halle 4/5 Schrankfertigung [Wh]                0
2.2 - Halle 2/4 Rilsan [Wh]                          0
2.3 - Halle 4/5 Stahlstuhl [Wh]                      0
2.4 - Halle 4/1 Schichtholz [Wh]                     0
2.5 - Halle 4/1 Absaugung Schichtholz [Wh]           0
dtype: int64

Indices of outliers:


In [ ]:
print(ENIT_df.shape)
ENIT_df.head()

In [ ]:
ENIT_df['Übergabezähler [Wh]'].unique()

array([    0.  , 20000.  , 40000.  , ..., 66666.67, 86666.67, 73333.33])

In [ ]:
# NA values
print(ENIT_df.isna().sum())

In [ ]:
# data as dict with each company as a key
FlexGuideData = {
    'MechTron': ENIT_df
}

# pre-processing pipeline

In [11]:
def outlier_detection(data, dependent_var, independent_var):
    # only keep demand and one other variable
    df = data[[f'{dependent_var}', f'{independent_var}']]

    # check for outliers
    # Calculate the mean and standard deviation for each column
    mean = df.mean()
    std_dev = df.std()

    # Identify outliers using twice the standard deviation
    outliers = (df < (mean - 3 * std_dev)) | (df > (mean + 3 * std_dev))

    # Print the number of True values in each column
    print("Number of outliers in each column:")
    print(outliers.sum())

    # Get the row and column indices of the True values
    true_values_indices = outliers.where(outliers).stack().index

    # Print the row and column indices of the True values
    print("\nIndices of outliers:")
    for row, col in true_values_indices:
        print(f"Row: {row}, Column: {col}")

    return df

In [12]:
# column names as strings
Kelmarsh_1 = outlier_detection(Kelmarsh_df['1'], 'Energy Export (kWh)', 'Long Term Wind (m/s)')

Number of outliers in each column:
Energy Export (kWh)     6
Long Term Wind (m/s)    0
dtype: int64

Indices of outliers:
Row: 27407, Column: Energy Export (kWh)
Row: 10456, Column: Energy Export (kWh)
Row: 17788, Column: Energy Export (kWh)
Row: 24557, Column: Energy Export (kWh)
Row: 26026, Column: Energy Export (kWh)
Row: 37678, Column: Energy Export (kWh)


In [23]:
%store Kelmarsh_df Penmanshiel_df

Stored 'Kelmarsh_df' (defaultdict)
Stored 'Penmanshiel_df' (defaultdict)
